In [2]:
from bs4 import BeautifulSoup
from docx import Document
import pandas as pd

# Ruta del archivo
file_path = r"D:\Documents\Scrapping Sentimientos Project\Project_repository\Docs Mercadolibre\Apple iPhone 15 (128 GB) - Negro $3217900.docx"

# Leer el contenido del documento .docx
doc = Document(file_path)
html_content = "\n".join([paragraph.text for paragraph in doc.paragraphs])

# Analizar el contenido HTML con BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Extraer los comentarios
comments = soup.find_all('p', class_='ui-review-capability-comments__comment__content')
extracted_comments = [comment.get_text(strip=True) for comment in comments]

# Extraer las calificaciones
ratings = []
rating_containers = soup.find_all('div', class_='ui-review-capability-comments__comment__rating')
for container in rating_containers:
    stars = len(container.find_all('use', href='#poly_star_fill'))
    ratings.append(stars)

# Crear un DataFrame con los comentarios y las calificaciones
df = pd.DataFrame({'Comments': extracted_comments, 'Rating': ratings})

# Mostrar el DataFrame
print(df)

                                              Comments  Rating
0    El producto es nuevo. Pero es versión china. I...       5
1    Después de un mes de uso constante puedo decir...       5
2    Hasta ahora todo bien. Al menos el modelo que ...       5
3    Es la versión china, totalmente original pero ...       5
4    Fueron muchas dudas y miedo para decidirme a l...       5
..                                                 ...     ...
125  A pesar de que es un dispositivo original y nu...       1
126  Todo iba bien, configure el celular mientras m...       2
127  Al momento de registrar el equipo sale inegi d...       3
128                                             Bueno.       3
129                            No suministran factura.       1

[130 rows x 2 columns]


In [ ]:
###########################  NOTEBOOK JUAN PABLO ##############################

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

from selenium.webdriver.chrome.options import Options

# Configuración para simular un navegador real
chrome_options = Options()

# Evitar ser detectado por automatización
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--incognito")

# Agregar un user-agent personalizado
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.199 Safari/537.36"
)

# Inicializar el driver con las opciones configuradas
driver_path = 'D:\Documents\Scrapping Sentimientos Project\Project_repository\chromedriver.exe'
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# URL de la página
url = 'https://www.mercadolibre.com.co/xiaomi-redmi-note-13-pro-4g-dual-sim-256-gb-purpura-8-gb-ram/p/MCO35734320'
driver.get(url)

# Espera inicial para cargar la página
time.sleep(5)

# Haz clic en el botón "Mostrar todas las opiniones" para abrir el modal
try:
    load_more_button = driver.find_element(By.CSS_SELECTOR, 'button.show-more-click[data-testid="see-more"]')
    driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
    time.sleep(1)
    driver.execute_script("arguments[0].click();", load_more_button)  # Haz clic en el botón
    time.sleep(5)  # Ajustar el tiempo si es necesario
    print("Botón 'Mostrar todas las opiniones' clicado.")
except NoSuchElementException:
    print("No se encontró el botón para abrir los comentarios.")
    driver.quit()
    exit()

# Espera a que el modal de comentarios esté completamente cargado
try:
    modal_container = WebDriverWait(driver, 20).until(  # Incrementa el tiempo de espera
        EC.presence_of_element_located((By.CSS_SELECTOR, '.andes-modal__content'))
    )
    print("Modal de comentarios encontrado.")
except TimeoutException:
    print("El modal de comentarios no se cargó a tiempo. Guardando el HTML completo para depuración.")
    # Guardar el HTML completo para inspección
    with open("page_debug.html", "w", encoding="utf-8") as file:
        file.write(driver.page_source)

    driver.quit()
    exit()

# Forzar el scroll dentro del modal
scroll_pause_time = 2  # Tiempo de espera entre cada scroll

try:
    last_height = driver.execute_script("return arguments[0].scrollHeight;", modal_container)
    while True:
        # Ejecuta el scroll hacia el final
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", modal_container)
        time.sleep(scroll_pause_time)
        
        # Verifica si la altura ya no cambia
        new_height = driver.execute_script("return arguments[0].scrollHeight;", modal_container)
        if new_height == last_height:
            print("Se alcanzó el final del modal.")
            break
        last_height = new_height
except Exception as e:
    print(f"Error durante el scroll: {e}")
    driver.quit()
    exit()

# Extrae los comentarios con BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')
comment_elements = soup.select('p.ui-review-capability-comments__comment__content')  # Selector para comentarios en el modal
comments = [comment.get_text(strip=True) for comment in comment_elements]

# Imprime los comentarios extraídos
if comments:
    for idx, comment in enumerate(comments, 1):
        print(f"Comentario {idx}: {comment}")
else:
    print("No se encontraron comentarios con el selector actual.")

# Guardar los comentarios en un archivo para inspección
with open("comentarios_extraidos.txt", "w", encoding="utf-8") as file:
    for comment in comments:
        file.write(comment + "\n")

# Cierra el navegador
driver.quit()


Botón 'Mostrar todas las opiniones' clicado.
Modal de comentarios encontrado.
Se alcanzó el final del modal.
Comentario 1: Está bueno y funciona bien, esperemos que no salga chimbo, como mi ex.
Comentario 2: Me parece un movil muy bonito, tiene un duracion de bateria excelente.
Comentario 3: Buen equipo, vengo de usar un redmi note 9, y vaya la diferencia, bastante fluido, transiciones suaves, buen espacio, recomendado si no quieres hacer un gasto tan grande y tener buenas prestaciones.


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

from selenium.webdriver.chrome.options import Options

# Configuración para simular un navegador real
chrome_options = Options()

# Evitar ser detectado por automatización
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--incognito")

# Agregar un user-agent personalizado
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.199 Safari/537.36"
)

# Inicializar el driver con las opciones configuradas
driver_path = 'D:\Documents\Scrapping Sentimientos Project\Project_repository\chromedriver.exe'
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# URL de la página
url = 'https://www.mercadolibre.com.co/xiaomi-redmi-note-13-pro-4g-dual-sim-256-gb-purpura-8-gb-ram/p/MCO35734320'
driver.get(url)

# Espera inicial para cargar la página
time.sleep(5)

# Haz clic en el botón "Mostrar todas las opiniones" para abrir el modal
try:
    load_more_button = driver.find_element(By.CSS_SELECTOR, 'button.show-more-click[data-testid="see-more"]')
    driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
    time.sleep(1)
    driver.execute_script("arguments[0].click();", load_more_button)  # Haz clic en el botón
    time.sleep(5)  # Ajustar el tiempo si es necesario
    print("Botón 'Mostrar todas las opiniones' clicado.")
except NoSuchElementException:
    print("No se encontró el botón para abrir los comentarios.")
    driver.quit()
    exit()

# Espera a que el contenedor de comentarios esté completamente cargado
try:
    modal_container = WebDriverWait(driver, 20).until(  # Incrementa el tiempo de espera
        EC.presence_of_element_located((By.CSS_SELECTOR, 'section[data-testid="reviews-desktop"].ui-review-capability'))
    )
    print("Contenedor de comentarios encontrado.")
except TimeoutException:
    print("El contenedor de comentarios no se cargó a tiempo. Guardando el HTML completo para depuración.")
    # Guardar el HTML completo para inspección
    with open("page_debug.html", "w", encoding="utf-8") as file:
        file.write(driver.page_source)

    driver.quit()
    exit()

# Forzar el scroll dentro del contenedor
scroll_pause_time = 2  # Tiempo de espera entre cada scroll

try:
    last_height = driver.execute_script("return arguments[0].scrollHeight;", modal_container)
    while True:
        # Ejecuta el scroll hacia el final
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", modal_container)
        time.sleep(scroll_pause_time)
        
        # Verifica si la altura ya no cambia
        new_height = driver.execute_script("return arguments[0].scrollHeight;", modal_container)
        if new_height == last_height:
            print("Se alcanzó el final del contenedor.")
            break
        last_height = new_height
except Exception as e:
    print(f"Error durante el scroll: {e}")
    driver.quit()
    exit()

# Extrae los comentarios con BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')
comment_elements = soup.select('p.ui-review-capability-comments__comment__content')  # Ajusta el selector si es necesario
comments = [comment.get_text(strip=True) for comment in comment_elements]

# Imprime los comentarios extraídos
if comments:
    for idx, comment in enumerate(comments, 1):
        print(f"Comentario {idx}: {comment}")
else:
    print("No se encontraron comentarios con el selector actual.")

# Guardar los comentarios en un archivo para inspección
with open("comentarios_extraidos.txt", "w", encoding="utf-8") as file:
    for comment in comments:
        file.write(comment + "\n")

# Cierra el navegador
driver.quit()

Botón 'Mostrar todas las opiniones' clicado.
El contenedor de comentarios no se cargó a tiempo. Guardando el HTML completo para depuración.
Error durante el scroll: HTTPConnectionPool(host='localhost', port=52466): Max retries exceeded with url: /session/fabda741f09cb2c0efa549b0ab8af3ac/execute/sync (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000187FB4BBA60>: Failed to establish a new connection: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión'))


MaxRetryError: HTTPConnectionPool(host='localhost', port=52466): Max retries exceeded with url: /session/fabda741f09cb2c0efa549b0ab8af3ac/source (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000187FB4BBA00>: Failed to establish a new connection: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión'))

: 

In [ ]:
############################ FINAL DE VARIOS DOC WORD ############################33

In [3]:
import os
from bs4 import BeautifulSoup
from docx import Document
import pandas as pd

# Ruta de la carpeta que contiene los documentos
folder_path = r"D:\Documents\Scrapping Sentimientos Project\Project_repository\Docs Mercadolibre"

# Lista para almacenar los datos de todos los archivos
all_data = []

# Iterar sobre todos los archivos en la carpeta
for filename in os.listdir(folder_path):
    if filename.endswith('.docx'):  # Verificar si el archivo es .docx
        file_path = os.path.join(folder_path, filename)

        # Leer el contenido del documento .docx
        doc = Document(file_path)
        html_content = "\n".join([paragraph.text for paragraph in doc.paragraphs])

        # Analizar el contenido HTML con BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        # Extraer los comentarios
        comments = soup.find_all('p', class_='ui-review-capability-comments__comment__content')
        extracted_comments = [comment.get_text(strip=True) for comment in comments]

        # Extraer las calificaciones
        ratings = []
        rating_containers = soup.find_all('div', class_='ui-review-capability-comments__comment__rating')
        for container in rating_containers:
            stars = len(container.find_all('use', href='#poly_star_fill'))
            ratings.append(stars)

        # Asegurar que comentarios y calificaciones tengan el mismo tamaño
        if len(extracted_comments) > len(ratings):
            ratings.extend([None] * (len(extracted_comments) - len(ratings)))
        elif len(ratings) > len(extracted_comments):
            extracted_comments.extend([None] * (len(ratings) - len(extracted_comments)))

        # Agregar los datos al conjunto general
        for comment, rating in zip(extracted_comments, ratings):
            all_data.append({'Filename': filename, 'Comment': comment, 'Rating': rating})

# Crear un DataFrame con todos los datos
df = pd.DataFrame(all_data)

# Guardar el DataFrame en un archivo CSV (opcional)
df.to_csv('all_comments_and_ratings.csv', index=False)

# Mostrar el DataFrame
print(df)

                                               Filename  \
0        Apple iPhone 11 (128 GB) - Negro $1754910.docx   
1        Apple iPhone 11 (128 GB) - Negro $1754910.docx   
2        Apple iPhone 11 (128 GB) - Negro $1754910.docx   
3        Apple iPhone 11 (128 GB) - Negro $1754910.docx   
4        Apple iPhone 11 (128 GB) - Negro $1754910.docx   
...                                                 ...   
2494  Xiaomi Redmi Note 13 Pro+ 5G Dual SIM 512 GB N...   
2495  Xiaomi Redmi Note 13 Pro+ 5G Dual SIM 512 GB N...   
2496  Xiaomi Redmi Note 13 Pro+ 5G Dual SIM 512 GB N...   
2497  Xiaomi Redmi Note 13 Pro+ 5G Dual SIM 512 GB N...   
2498  Xiaomi Redmi Note 13 Pro+ 5G Dual SIM 512 GB N...   

                                                Comment  Rating  
0     Es un dispositivo magnifico, se adapta a la ne...       5  
1     Excelente precio para un equipo tan completo, ...       5  
2                    No sabía que llegaba sin cargador.       4  
3     Ha sido excelente est

In [8]:
# Agrupar por "Filename" y contar los comentarios únicos
grouped_comments = df.groupby('Filename')['Comment'].count()

print(grouped_comments)

Filename
Apple iPhone 11 (128 GB) - Negro $1754910.docx                                         83
Apple iPhone 13 (128 GB) - Azul $2499900.docx                                         180
Apple iPhone 14 (128 GB) - Medianoche $3169000.docx                                    50
Apple iPhone 15 (128 GB) - Negro $3217900.docx                                        130
Samsung A05s 4G Dual SIM 128 GB Negro 6 GB RAM $431433.docx                            90
Samsung Galaxy A15 4G Dual SIM 256 GB azul oscuro 8 GB RAM $599900.docx               173
Samsung Galaxy A25 5G 5G 128 GB azul 6 GB RAM $849900.docx                             71
Samsung Galaxy A35 5g 256 GB Azul oscuro 8 GB RAM $949900.docx                        106
Samsung Galaxy A55 5G Dual SIM 256 GB azul oscuro 8 GB RAM $1387900.docx              210
Samsung Galaxy S23 Ultra (eSIM) 5G Dual SIM 512 GB green 12 GB RAM $4349900.docx      210
Samsung Galaxy S24 Ultra 5G Dual SIM 512 GB titanium black 12 GB RAM $4529900.docx    111
X

In [ ]:
Samsung A05s 4G Dual SIM 128 GB Negro 6 GB RAM $431433
Xiaomi Redmi A3 Dual SIM 64 GB midnight black 3 GB RAM $299900.docx